# Hospital Readmission Prediction Demo with XGBoost and SHAP

This notebook demonstrates how to:
1. Generate synthetic patient readmission data
2. Train an XGBoost model to predict readmission risk
3. Evaluate model performance
4. Interpret predictions using SHAP values
5. Run unit tests to validate the model pipeline

The goal is to predict which patients are likely to be readmitted within 30 days of discharge.

## 1. Setup Dependencies

First, we'll import all required libraries. Make sure you have installed:
```bash
pip install xgboost shap pytest scikit-learn pandas numpy matplotlib
```

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pytest

# Set random seed for reproducibility
RANDOM_STATE = 42

## 2. Generate Synthetic Patient Data

We'll create a synthetic dataset that mimics real patient data with features like:
- Demographics (age, sex)
- Prior utilization (admissions, ED visits)
- Clinical factors (Charlson score, lab flags, medication count)
- Length of stay

In [ ]:
def generate_synthetic_data(n=5000, seed=RANDOM_STATE):
    rng = np.random.RandomState(seed)
    
    # Generate features
    data = {
        'age': rng.normal(65, 16, n).clip(18, 95).astype(int),
        'sex': rng.binomial(1, 0.52, n),  # 1=male, 0=female
        'prior_adm_6m': rng.poisson(0.6, n),
        'ed_visits_6m': rng.poisson(0.8, n),
        'charlson': rng.poisson(2.0, n),
        'labs_flag': rng.binomial(1, 0.15, n),
        'med_count': rng.poisson(5, n),
        'length_of_stay': rng.exponential(3.0, n).clip(1, 30)
    }
    
    # Calculate risk score and generate label
    risk_score = (
        0.02 * (data['age'] - 65) +
        0.6 * (data['prior_adm_6m'] > 0) +
        0.4 * (data['ed_visits_6m'] > 1) +
        0.3 * data['charlson'] / (data['charlson'].max() + 1) +
        0.05 * data['med_count'] +
        0.1 * (data['length_of_stay'] > 5) +
        rng.normal(0, 0.6, n)
    )
    
    # Convert to probability and binary outcome
    prob = 1 / (1 + np.exp(-risk_score))
    data['readmit_30d'] = (rng.uniform(0, 1, n) < prob * 0.4).astype(int)
    
    return pd.DataFrame(data)

# Generate dataset
df = generate_synthetic_data()
print("Dataset shape:", df.shape)
print("\nFeature summary:")
print(df.describe().round(2))
print("\nReadmission rate:", df['readmit_30d'].mean().round(3))

## 3. Prepare Data for Modeling

Split the data into training and test sets, then scale the features.

In [ ]:
# Split features and target
X = df.drop(columns=['readmit_30d'])
y = df['readmit_30d']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
print("Readmission rate in train:", y_train.mean().round(3))
print("Readmission rate in test:", y_test.mean().round(3))

## 4. Train XGBoost Model

We'll use XGBoost with parameters tuned for interpretability (limited depth) while maintaining good performance.

In [ ]:
# Initialize XGBoost classifier
xgb_params = {
    'max_depth': 4,  # Limit depth for interpretability
    'learning_rate': 0.1,
    'n_estimators': 100,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'random_state': RANDOM_STATE
}

model = xgb.XGBClassifier(**xgb_params)

# Train the model
model.fit(
    X_train_scaled, 
    y_train,
    eval_set=[(X_test_scaled, y_test)],
    eval_metric='auc',
    early_stopping_rounds=10,
    verbose=False
)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Show confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

## 5. SHAP Explanations

Calculate SHAP values to understand feature importance and their impact on predictions.

In [ ]:
# Calculate SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_scaled)

# Create SHAP summary plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test_scaled, feature_names=X.columns, show=False)
plt.title("SHAP Feature Importance")
plt.tight_layout()
plt.show()

# Show SHAP feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values).mean(0)
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

print("\nFeature Importance Ranking:")
print(feature_importance)

## 6. Individual Patient Explanations

Let's look at SHAP explanations for individual high-risk patients to understand what drives their predictions.

In [ ]:
# Get high-risk patients
high_risk_idx = np.where(y_pred_proba > 0.7)[0]
if len(high_risk_idx) > 0:
    # Plot SHAP values for first high-risk patient
    patient_idx = high_risk_idx[0]
    
    plt.figure(figsize=(10, 4))
    shap.force_plot(
        explainer.expected_value,
        shap_values[patient_idx,:],
        X_test_scaled[patient_idx,:],
        feature_names=X.columns,
        matplotlib=True,
        show=False
    )
    plt.title("SHAP Force Plot for High-Risk Patient")
    plt.tight_layout()
    plt.show()
    
    # Show patient details
    print("\nHigh-risk patient details:")
    patient_data = X_test.iloc[patient_idx]
    for feature, value in patient_data.items():
        print(f"{feature}: {value:.2f}")

## 7. Unit Tests

Here are some unit tests to validate our data generation and model pipeline.

In [ ]:
def test_data_generation():
    """Test that generated data meets expectations."""
    test_df = generate_synthetic_data(n=1000, seed=42)
    
    # Test shape
    assert test_df.shape == (1000, 9), "Wrong data shape"
    
    # Test value ranges
    assert test_df['age'].between(18, 95).all(), "Age out of range"
    assert test_df['sex'].isin([0, 1]).all(), "Invalid sex values"
    assert test_df['readmit_30d'].isin([0, 1]).all(), "Invalid target values"
    
    # Test reasonable readmission rate
    assert 0.05 <= test_df['readmit_30d'].mean() <= 0.35, "Unrealistic readmission rate"
    
    print("Data generation tests passed!")

def test_model_pipeline():
    """Test model training and predictions."""
    # Generate small dataset
    test_df = generate_synthetic_data(n=500, seed=42)
    X = test_df.drop(columns=['readmit_30d'])
    y = test_df['readmit_30d']
    
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Scale
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)
    
    # Train
    model = xgb.XGBClassifier(n_estimators=50, random_state=42)
    model.fit(X_train_s, y_train)
    
    # Predict
    y_pred = model.predict(X_test_s)
    y_pred_proba = model.predict_proba(X_test_s)
    
    # Tests
    assert y_pred.shape == y_test.shape, "Wrong prediction shape"
    assert np.all(y_pred_proba >= 0) and np.all(y_pred_proba <= 1), "Invalid probabilities"
    assert y_pred.dtype == y_test.dtype, "Wrong prediction type"
    
    print("Model pipeline tests passed!")

# Run tests
test_data_generation()
test_model_pipeline()